<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Estructura_Base__Selector_de_Instrumento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === Librerías Base ===
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
from datetime import datetime
import base64

# === Estilo general del Dashboard ===
display(HTML("""
<style>
h3 { color: #1866a3; margin-top: 15px; }
.widget-label { font-weight: bold; }
</style>
"""))

# === Selector Maestro de Instrumento ===
instrumento_dropdown_maestro = widgets.Dropdown(
    options=[
        "Seleccionar...",
        "Puntos Fijos",
        "Piezómetros Eléctricos",
        "Piezómetros Casagrande",
        "Celdas de Asentamiento",
        "Freatímetros",
        "Extensómetros"
    ],
    value="Seleccionar...",
    description="Instrumento:",
    layout=widgets.Layout(width='320px')
)

# Contenedor principal donde se cargará dinámicamente el widget del instrumento seleccionado
output_maestro = widgets.Output()

# === Función para cambiar de instrumento dinámicamente ===
def cargar_instrumento(change):
    with output_maestro:
        clear_output(wait=True)
        seleccion = instrumento_dropdown_maestro.value

        if seleccion == "Seleccionar...":
            display(HTML("<b style='color:gray'>Selecciona un instrumento para comenzar.</b>"))

        elif seleccion == "Puntos Fijos":
            display(HTML("<h3>📍 Puntos Fijos</h3>"))
            # Aquí se llamará al bloque del script de Puntos Fijos que ya tienes preparado.
            # Ejemplo:
            # ejecutar_script_puntos_fijos()
            # === Bloque: Puntos Fijos ===

            def ejecutar_script_puntos_fijos():
                clear_output(wait=True)

                display(HTML("<h3 style='color:#1866a3'>📍 Estado/Alarma - Puntos Fijos</h3>"))

                # Selectores
                origen_dropdown_pf = widgets.Dropdown(
                    options=["CSV", "XLSX"],
                    value="CSV",
                    description="Origen:",
                    layout=widgets.Layout(width='160px')
                )
                margen_dropdown_pf = widgets.Dropdown(description="Margen:", layout=widgets.Layout(width='160px'))
                punto_dropdown_pf = widgets.Dropdown(description="Punto Fijo:", layout=widgets.Layout(width='180px'))
                anio_pf_dropdown = widgets.Dropdown(description="Año:", layout=widgets.Layout(width='120px'))
                filas_dropdown_pf = widgets.Dropdown(
                    options=[10, 20, 50, 100, 500, 1000, 'Todos'],
                    value=20,
                    description='Filas:',
                    layout=widgets.Layout(width='120px')
                )

                slider_widgets_pf = {}
                sliders_box_pf = widgets.VBox()

                # Crear sliders
                def crear_slider_con_botones_pf(col, col_min, col_max, col_range):
                    slider = widgets.FloatSlider(
                        value=0.0,
                        min=np.floor(col_min - 0.2 * col_range),
                        max=np.ceil(col_max + 0.2 * col_range),
                        step=0.1,
                        description=f"{col}:",
                        continuous_update=False,
                        readout=True,
                        readout_format='.2f',
                        layout=widgets.Layout(width='350px'),
                        style={'description_width': '180px'}
                    )
                    menos_btn = widgets.Button(description='-', layout=widgets.Layout(width='32px'))
                    mas_btn = widgets.Button(description='+', layout=widgets.Layout(width='32px'))

                    def menos_clicked(b):
                        slider.value = max(slider.value - slider.step, slider.min)
                        slider.value = np.round(slider.value, 2)

                    def mas_clicked(b):
                        slider.value = min(slider.value + slider.step, slider.max)
                        slider.value = np.round(slider.value, 2)

                    menos_btn.on_click(menos_clicked)
                    mas_btn.on_click(mas_clicked)

                    return slider, widgets.HBox([slider, menos_btn, mas_btn])

                def crear_sliders_pf(df):
                    global slider_widgets_pf
                    slider_widgets_pf = {}
                    sliders = []
                    incluir = ['DISTANCIA_(MM)', 'TASA_DISTANCIA_(MM/DIA)', 'AZIMUT_(GRADOS)']
                    for col in incluir:
                        if col in df.columns:
                            col_numeric = pd.to_numeric(df[col], errors='coerce')
                            col_min = col_numeric.min(skipna=True) if not col_numeric.dropna().empty else 0.0
                            col_max = col_numeric.max(skipna=True) if not col_numeric.dropna().empty else col_min + 1.0
                            col_range = col_max - col_min if col_max > col_min else 1.0
                            slider, hbox = crear_slider_con_botones_pf(col, col_min, col_max, col_range)
                            slider_widgets_pf[col] = slider
                            sliders.append(hbox)
                    sliders_box_pf.children = sliders

                def obtener_df_actual_pf():
                    origen = origen_dropdown_pf.value
                    return (datos_csv.get("puntos_fijos", pd.DataFrame()) if origen == "CSV" else datos_xlsx.get("puntos_fijos", pd.DataFrame())).copy()

                def actualizar_puntos_pf(change=None):
                    df = obtener_df_actual_pf()
                    if df.empty:
                        punto_dropdown_pf.options = []
                        return
                    margen = margen_dropdown_pf.value
                    if margen == "Margen Izquierda (MI)":
                        puntos = sorted(df[df['MARGEN'] == 'MI']['INSTRUMENTO'].dropna().unique())
                    elif margen == "Margen Derecha (MD)":
                        puntos = sorted(df[df['MARGEN'] == 'MD']['INSTRUMENTO'].dropna().unique())
                    else:
                        puntos = sorted(df['INSTRUMENTO'].dropna().unique())
                    punto_dropdown_pf.options = ["Todos"] + puntos
                    punto_dropdown_pf.value = "Todos"

                def actualizar_opciones_pf(change=None):
                    df = obtener_df_actual_pf()
                    if df.empty:
                        margen_dropdown_pf.options = []
                        punto_dropdown_pf.options = []
                        anio_pf_dropdown.options = []
                        sliders_box_pf.children = []
                        return
                    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                    margen_dropdown_pf.options = ["Margen Izquierda (MI)", "Margen Derecha (MD)"]
                    margen_dropdown_pf.value = "Margen Izquierda (MI)"
                    actualizar_puntos_pf()
                    anios = sorted(df['FECHA'].dt.year.dropna().unique())
                    anio_pf_dropdown.options = ["Todos"] + [str(a) for a in anios]
                    anio_pf_dropdown.value = "Todos"
                    crear_sliders_pf(df)

                margen_dropdown_pf.observe(actualizar_puntos_pf, names='value')
                origen_dropdown_pf.observe(actualizar_opciones_pf, names='value')
                actualizar_opciones_pf()

                tipos_alarma_pf = [("Todos", "todos"), ("Normal 🟢", "normal"), ("Alerta 🟡", "alerta"), ("Peligro 🔴", "peligro"), ("Sin dato ⚪️", "sin dato")]
                alarma_selector_pf = widgets.Dropdown(
                    options=tipos_alarma_pf, value="todos",
                    description="Ver alarmas:", layout=widgets.Layout(width='160px')
                )
                boton_estado_pf = widgets.Button(description="Calcular estado", button_style="warning", layout=widgets.Layout(width='150px'))
                boton_descargar_pf = widgets.Button(description="📥 Descargar XLSX", button_style="success", layout=widgets.Layout(width='180px'))

                output_estado_pf = widgets.Output()
                output_tabla_pf = widgets.Output()
                output_piechart_pf = widgets.Output()
                output_descarga_pf = widgets.Output()

                # Aquí puedes reutilizar las funciones de clasificación, emoji_alarma, mostrar_tabla y descargar
                # que ya tienes definidas para Puntos Fijos, pegándolas aquí.

                # Finalmente:
                pf_controles = widgets.VBox([
                    widgets.HBox([origen_dropdown_pf, margen_dropdown_pf, punto_dropdown_pf, anio_pf_dropdown]),
                    widgets.HTML("<b>Ajustá los umbrales para clasificar alarmas:</b>"),
                    sliders_box_pf,
                    widgets.HBox([filas_dropdown_pf, alarma_selector_pf, boton_estado_pf, boton_descargar_pf]),
                    output_descarga_pf
                ])

                display(pf_controles, output_estado_pf, output_tabla_pf, output_piechart_pf)



        elif seleccion == "Piezómetros Eléctricos":
            display(HTML("<h3>💧 Piezómetros Eléctricos</h3>"))
            # ejecutar_script_piezometros_electricos()
            # === Bloque: Piezómetros Eléctricos ===

            def ejecutar_script_piezometros_electricos():
                clear_output(wait=True)

                display(HTML("<h3 style='color:#1866a3'>💧 Estado/Alarma - Piezómetros Eléctricos</h3>"))

                # Selectores
                origen_dropdown_pe = widgets.Dropdown(
                    options=["CSV", "XLSX"],
                    value="CSV",
                    description="Origen:",
                    layout=widgets.Layout(width='160px')
                )
                progresiva_dropdown_pe = widgets.Dropdown(description="Progresiva:", layout=widgets.Layout(width='180px'))
                piezometro_dropdown_pe = widgets.Dropdown(description="Piezómetro:", layout=widgets.Layout(width='180px'))
                anio_pe_dropdown = widgets.Dropdown(description="Año:", layout=widgets.Layout(width='120px'))
                filas_dropdown_pe = widgets.Dropdown(
                    options=[10, 20, 50, 100, 500, 1000, 'Todos'],
                    value=20,
                    description='Filas:',
                    layout=widgets.Layout(width='120px')
                )

                slider_widgets_pe = {}
                sliders_box_pe = widgets.VBox()

                # Crear sliders
                def crear_slider_con_botones_pe(col, col_min, col_max, col_range):
                    slider = widgets.FloatSlider(
                        value=0.0,
                        min=np.floor(col_min - 0.2 * col_range),
                        max=np.ceil(col_max + 0.2 * col_range),
                        step=0.1,
                        description=f"{col}:",
                        continuous_update=False,
                        readout=True,
                        readout_format='.2f',
                        layout=widgets.Layout(width='350px'),
                        style={'description_width': '180px'}
                    )
                    menos_btn = widgets.Button(description='-', layout=widgets.Layout(width='32px'))
                    mas_btn = widgets.Button(description='+', layout=widgets.Layout(width='32px'))

                    def menos_clicked(b):
                        slider.value = max(slider.value - slider.step, slider.min)
                        slider.value = np.round(slider.value, 2)

                    def mas_clicked(b):
                        slider.value = min(slider.value + slider.step, slider.max)
                        slider.value = np.round(slider.value, 2)

                    menos_btn.on_click(menos_clicked)
                    mas_btn.on_click(mas_clicked)

                    return slider, widgets.HBox([slider, menos_btn, mas_btn])

                def crear_sliders_pe(df):
                    global slider_widgets_pe
                    slider_widgets_pe = {}
                    sliders = []
                    incluir = ['COTA_RIO_(M.S.N.M)', 'MCA_2_(FACTOR_G_Y_K)', 'COTA_NF']
                    for col in incluir:
                        if col in df.columns:
                            col_numeric = pd.to_numeric(df[col], errors='coerce')
                            col_min = col_numeric.min(skipna=True) if not col_numeric.dropna().empty else 0.0
                            col_max = col_numeric.max(skipna=True) if not col_numeric.dropna().empty else col_min + 1.0
                            col_range = col_max - col_min if col_max > col_min else 1.0
                            slider, hbox = crear_slider_con_botones_pe(col, col_min, col_max, col_range)
                            slider_widgets_pe[col] = slider
                            sliders.append(hbox)
                    sliders_box_pe.children = sliders

                def obtener_df_actual_pe():
                    origen = origen_dropdown_pe.value
                    return (datos_csv.get("piezometros_electricos", pd.DataFrame()) if origen == "CSV" else datos_xlsx.get("piezometros_electricos", pd.DataFrame())).copy()

                def actualizar_piezometros_pe(change=None):
                    df = obtener_df_actual_pe()
                    if df.empty:
                        piezometro_dropdown_pe.options = []
                        return
                    prog = progresiva_dropdown_pe.value
                    if prog == "Todas":
                        piezos = sorted(df['PIEZOMETRO'].dropna().unique())
                    else:
                        piezos = sorted(df[df['PROGRESIVA'] == prog]['PIEZOMETRO'].dropna().unique())
                    piezometro_dropdown_pe.options = ["Todos"] + piezos
                    piezometro_dropdown_pe.value = "Todos"

                def actualizar_opciones_pe(change=None):
                    df = obtener_df_actual_pe()
                    if df.empty:
                        progresiva_dropdown_pe.options = []
                        piezometro_dropdown_pe.options = []
                        anio_pe_dropdown.options = []
                        sliders_box_pe.children = []
                        return
                    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                    progresivas = sorted(df['PROGRESIVA'].dropna().unique())
                    progresiva_dropdown_pe.options = ["Todas"] + list(progresivas)
                    progresiva_dropdown_pe.value = "Todas"
                    actualizar_piezometros_pe()
                    anios = sorted(df['FECHA'].dt.year.dropna().unique())
                    anio_pe_dropdown.options = ["Todos"] + [str(a) for a in anios]
                    anio_pe_dropdown.value = "Todos"
                    crear_sliders_pe(df)

                progresiva_dropdown_pe.observe(actualizar_piezometros_pe, names='value')
                origen_dropdown_pe.observe(actualizar_opciones_pe, names='value')
                actualizar_opciones_pe()

                tipos_alarma_pe = [("Todos", "todos"), ("Normal 🟢", "normal"), ("Alerta 🟡", "alerta"), ("Peligro 🔴", "peligro"), ("Sin dato ⚪️", "sin dato")]
                alarma_selector_pe = widgets.Dropdown(
                    options=tipos_alarma_pe, value="todos",
                    description="Ver alarmas:", layout=widgets.Layout(width='160px')
                )
                boton_estado_pe = widgets.Button(description="Calcular estado", button_style="warning", layout=widgets.Layout(width='150px'))
                boton_descargar_pe = widgets.Button(description="📥 Descargar XLSX", button_style="success", layout=widgets.Layout(width='180px'))

                output_estado_pe = widgets.Output()
                output_tabla_pe = widgets.Output()
                output_piechart_pe = widgets.Output()
                output_descarga_pe = widgets.Output()

                # Aquí puedes reutilizar las funciones de clasificación, emoji_alarma, mostrar_tabla y descargar
                # que ya tienes definidas para Piezómetros Eléctricos, pegándolas aquí.

                pe_controles = widgets.VBox([
                    widgets.HBox([origen_dropdown_pe, progresiva_dropdown_pe, piezometro_dropdown_pe, anio_pe_dropdown]),
                    widgets.HTML("<b>Ajustá los umbrales para clasificar alarmas:</b>"),
                    sliders_box_pe,
                    widgets.HBox([filas_dropdown_pe, alarma_selector_pe, boton_estado_pe, boton_descargar_pe]),
                    output_descarga_pe
                ])

                display(pe_controles, output_estado_pe, output_tabla_pe, output_piechart_pe)


        elif seleccion == "Piezómetros Casagrande":
            display(HTML("<h3>💧 Piezómetros Casagrande</h3>"))
            # ejecutar_script_piezometros_casagrande()

        elif seleccion == "Celdas de Asentamiento":
            display(HTML("<h3>📏 Celdas de Asentamiento</h3>"))
            # ejecutar_script_celdas_asentamiento()

        elif seleccion == "Freatímetros":
            display(HTML("<h3>🌊 Freatímetros</h3>"))
            # ejecutar_script_freatimetros()

        elif seleccion == "Extensómetros":
            display(HTML("<h3>📐 Extensómetros</h3>"))
            # ejecutar_script_extensometros()

        else:
            display(HTML("<b style='color:red'>Instrumento no reconocido.</b>"))

# Conectar el selector maestro con la función
instrumento_dropdown_maestro.observe(cargar_instrumento, names='value')

# === Display inicial ===
display(widgets.HTML("<h3>🛠️ Dashboard Maestro Estado/Alarma – Instrumentos de Auscultación</h3>"))
display(instrumento_dropdown_maestro)
display(output_maestro)


SyntaxError: invalid syntax (ipython-input-1-1831892229.py, line 194)